<a href="https://colab.research.google.com/github/Hamza-333/Image_Segmentation/blob/main/CSC420_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models, torchvision.datasets
from torch.utils.data import Dataset
from torchvision.io import read_image
import os
import shutil
import random

# Data Preparation

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# !ls gdrive/MyDrive/CSC420_Final_Project/project3/images

In [3]:
example_clothing = read_image("/content/gdrive/MyDrive/CSC420_Final_Project/project3/labels_clothing/0001_clothes.png")
example_person = read_image("/content/gdrive/MyDrive/CSC420_Final_Project/project3/labels_person/0001_person.png")

In [4]:
example_img = read_image("/content/gdrive/MyDrive/CSC420_Final_Project/project3/images/0001.jpg")

In [5]:
# Move images to validation and test folders
def move_images(num_imgs, dst_imgs_dir, dst_labels_c_dir, dst_labels_p_dir, src_imgs_dir, src_labels_c_dir, src_labels_p_dir):

    src_imgs = sorted(os.listdir(src_imgs_dir))
    src_c_labels = sorted(os.listdir(src_labels_c_dir))
    src_p_labels = sorted(os.listdir(src_labels_p_dir))
    if '.DS_Store' in src_imgs[0]:
        src_imgs = src_imgs[1:]
    if '.DS_Store' in src_c_labels[0]:
        src_c_labels = src_c_labels[1:]
    if '.DS_Store' in src_p_labels[0]:
        src_p_labels = src_p_labels[1:]

    if not os.path.exists(dst_imgs_dir):
        os.mkdir(dst_imgs_dir)
    if not os.path.exists(dst_labels_c_dir):
        os.mkdir(dst_labels_c_dir)
    if not os.path.exists(dst_labels_p_dir):
        os.mkdir(dst_labels_p_dir)

    random.seed(0)
    # print(random.sample(src_imgs, num_imgs))
    for img in random.sample(src_imgs, num_imgs):
        # move imgs and labels to destination folder
        src_imgs_path = os.path.join(src_imgs_dir, img)
        dst_imgs_path = os.path.join(dst_imgs_dir, img)
        shutil.move(src_imgs_path, dst_imgs_path)

    random.seed(0)
    # print(random.sample(src_c_labels, num_imgs))
    for img in random.sample(src_c_labels, num_imgs):
        src_labels_c_path = os.path.join(src_labels_c_dir, img)
        dst_labels_c_path = os.path.join(dst_labels_c_dir, img)
        shutil.move(src_labels_c_path, dst_labels_c_path)

    random.seed(0)
    # print(random.sample(src_p_labels, num_imgs))
    for img in random.sample(src_p_labels, num_imgs):
        src_labels_p_path = os.path.join(src_labels_p_dir, img)
        dst_labels_p_path = os.path.join(dst_labels_p_dir, img)
        shutil.move(src_labels_p_path, dst_labels_p_path)


In [6]:
val_dest_imgs = "/content/gdrive/MyDrive/CSC420_Final_Project/project3/val_images/"
val_dest_c_labels = "/content/gdrive/MyDrive/CSC420_Final_Project/project3/val_labels_clothing/"
val_dest_p_labels = "/content/gdrive/MyDrive/CSC420_Final_Project/project3/val_labels_person/"

test_dest_imgs = "/content/gdrive/MyDrive/CSC420_Final_Project/project3/test_images/"
test_dest_c_labels = "/content/gdrive/MyDrive/CSC420_Final_Project/project3/test_labels_clothing/"
test_dest_p_labels = "/content/gdrive/MyDrive/CSC420_Final_Project/project3/test_labels_person/"

src_imgs = "/content/gdrive/MyDrive/CSC420_Final_Project/project3/images/"
src_c_labels = "/content/gdrive/MyDrive/CSC420_Final_Project/project3/labels_clothing/"
src_p_labels = "/content/gdrive/MyDrive/CSC420_Final_Project/project3/labels_person/"

In [83]:
# move_images(60, val_dest_imgs, val_dest_c_labels, val_dest_p_labels, src_imgs, src_c_labels, src_p_labels)

In [84]:
# move_images(60, test_dest_imgs, test_dest_c_labels, test_dest_p_labels, src_imgs, src_c_labels, src_p_labels)

In [ ]:
print(example_img[0, 30, 100:])

In [ ]:
#example clothing label
print(example_clothing[0, 30, 100:])


In [ ]:
#example person label
print(example_person[0, 30, 100:])

In [13]:
print(len(os.listdir("/content/gdrive/MyDrive/CSC420_Final_Project/project3/val_labels_clothing/")))

60


In [7]:
import os

In [8]:
class SegmentationDataset(Dataset):
    def __init__(self, imgs_path, labels_path):
        self.imgs_path = imgs_path
        self.imgs = sorted(os.listdir(imgs_path))

        if '.DS_Store' in self.imgs[0]:
            self.imgs = self.imgs[1:]
        self.labels_path = labels_path

        self.labels = os.listdir(labels_path)
        if '.DS_Store' in self.labels[0]:
            self.labels = self.labels[1:]

    def __getitem__(self, idx):
        """ Get image and its corresponding label """
        # img_p = os.path.join(self.imgs_path, )
        # print(os.path.join(self.imgs_path, self.imgs[idx]))
        img = read_image(os.path.join(self.imgs_path, self.imgs[idx]))
        # label_p = self.labels_path[idx]
        # print(os.path.join(self.labels_path, self.labels[idx]))
        label = read_image(os.path.join(self.labels_path, self.labels[idx]))

        return (torch.tensor(img, dtype=torch.float32), torch.tensor(label, dtype=torch.float32))

    def __len__(self):
        return len(self.imgs_path)

In [9]:
train_data_person = SegmentationDataset("/content/gdrive/MyDrive/CSC420_Final_Project/project3/images/", "/content/gdrive/MyDrive/CSC420_Final_Project/project3/labels_person/")
train_data_clothing = SegmentationDataset("/content/gdrive/MyDrive/CSC420_Final_Project/project3/images/", "/content/gdrive/MyDrive/CSC420_Final_Project/project3/labels_clothing/")

val_data_person = SegmentationDataset("/content/gdrive/MyDrive/CSC420_Final_Project/project3/val_images/", "/content/gdrive/MyDrive/CSC420_Final_Project/project3/val_labels_person/")
val_data_clothing = SegmentationDataset("/content/gdrive/MyDrive/CSC420_Final_Project/project3/val_images/", "/content/gdrive/MyDrive/CSC420_Final_Project/project3/val_labels_clothing/")

test_data_person = SegmentationDataset("/content/gdrive/MyDrive/CSC420_Final_Project/project3/test_images/", "/content/gdrive/MyDrive/CSC420_Final_Project/project3/test_labels_person/")
test_data_clothing = SegmentationDataset("/content/gdrive/MyDrive/CSC420_Final_Project/project3/test_images/", "/content/gdrive/MyDrive/CSC420_Final_Project/project3/test_labels_clothing/")

In [51]:
len(sorted(os.listdir("/content/gdrive/MyDrive/CSC420_Final_Project/project3/labels_clothing/")))

601

In [16]:
print(len(train_data_person.imgs))
print(len(train_data_clothing.imgs))
print(len(val_data_person.imgs))
print(len(val_data_clothing.imgs))
print(len(test_data_person.imgs))
print(len(test_data_clothing.imgs))

480
480
60
60
60
60


In [109]:
train_data_clothing.__getitem__(1)[1].size()

<ipython-input-14-fe6acb125cd4>:23: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return (torch.tensor(img, dtype=torch.float32), torch.tensor(label, dtype=torch.float32))


torch.Size([1, 600, 400])

# Resnet Model

In [20]:
class ResidualBlockEncoder(nn.Module):
    expansion = 1
    def __init__(self, input_channels, out_channels, dilation, stride=1,
                 downsample=None):
        super(ResidualBlockEncoder, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=input_channels, out_channels=out_channels,\
                           kernel_size=3, stride=stride, padding=dilation, dilation=dilation)

        self.norm1 = nn.BatchNorm2d(num_features=out_channels)
        # self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        # 32 x 32 x 75 x 50
        self.conv2 = nn.Conv2d(in_channels=out_channels, out_channels=out_channels,\
                            kernel_size=3, stride=stride, padding=dilation, dilation=dilation)

        self.norm2 = nn.BatchNorm2d(num_features=out_channels)
        self.relu = nn.ReLU()
        self.downsample = downsample
    def forward(self, x):
        # print('Encoder')
        residual = x.clone()
        # print('Residual Before downsample:', residual.size())
        # print('X Before conv:', x.size())
        x = self.relu(self.norm1(self.conv1(x)))
        # print('X After conv1:', x.size())
        x = self.relu(self.norm2(self.conv2(x)))

        if self.downsample:
            residual = self.downsample(residual)

        # print('Residual After downsample:', residual.size())
        # print('X After conv:', x.size())
        x = x + residual
        x = self.relu(x)

        return x


In [11]:
class ResNetEncoder(nn.Module):
    def __init__(self, num_blocks, ResidualBlock, dilations, img_channels=3,
                 out_channels=16):
        super(ResNetEncoder, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=img_channels, out_channels=out_channels,\
                           kernel_size=3, stride=2, padding=1)
        self.norm1 = nn.BatchNorm2d(num_features=out_channels)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, return_indices=True)
        self.layer_in_channels = out_channels
        self.layer1 = self._make_layer(ResidualBlock, num_blocks[0], dilations[0], out_channels=16)
        self.layer2 = self._make_layer(ResidualBlock, num_blocks[1], dilations[1], out_channels=32, stride=1)
        self.layer3 = self._make_layer(ResidualBlock, num_blocks[2], dilations[2], out_channels=64, stride=1)
        self.layer4 = self._make_layer(ResidualBlock, num_blocks[3], dilations[3], out_channels=128, stride=1)

        self.relu = nn.ReLU()

    def _make_layer(self, ResidualBlock, num_blocks, dilation, out_channels, stride=1):
        downsample = None
        if stride != 1 or self.layer_in_channels != out_channels * ResidualBlock.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.layer_in_channels, out_channels * ResidualBlock.expansion, stride=stride, kernel_size=1, bias=False),
                nn.BatchNorm2d(out_channels * ResidualBlock.expansion)
            )

        layer = []
        layer.append(ResidualBlock(self.layer_in_channels, out_channels, dilation, stride, downsample))
        self.layer_in_channels = out_channels * ResidualBlock.expansion

        for i in range(1, num_blocks):
            layer.append(ResidualBlock(self.layer_in_channels, out_channels, dilation, stride))

        return nn.Sequential(*layer)

    def forward(self, x):

        x = self.relu(self.norm1(self.conv1(x)))
        x = self.layer1(x)
        p1_shape = x.shape
        x, ind_1 = self.pool(x)
        # x, ind_2 = self.pool(x)
        x = self.layer2(x)
        p2_shape = x.shape
        x, ind_2 = self.pool(x)

        x = self.layer3(x)
        p3_shape = x.shape
        x, ind_3 = self.pool(x)
        # x, ind_4 = self.pool(x)
        x = self.layer4(x)
        p4_shape = x.shape
        x, ind_4 = self.pool(x)
        # x, ind_5 = self.pool(x)

        # return x, ind_1, ind_2, ind_3, ind_4, ind_5
        return x, ind_1, ind_2, ind_3, ind_4, p1_shape, p2_shape, p3_shape, p4_shape

In [21]:
class ResidualBlockDecoder(nn.Module):
    expansion = 1
    def __init__(self, input_channels, out_channels, dilation, stride=1,
                 upsample=None):
        super(ResidualBlockDecoder, self).__init__()
        self.t_conv1 = nn.ConvTranspose2d(in_channels=input_channels, out_channels=out_channels,\
                           kernel_size=3, stride=stride, output_padding=0, dilation=dilation, padding=dilation)

        self.norm1 = nn.BatchNorm2d(num_features=out_channels)
        # self.pool = nn.MaxPool2d(kernel_size=2, stride=2)


        self.t_conv2 = nn.ConvTranspose2d(in_channels=out_channels, out_channels=out_channels,\
                            kernel_size=3, stride=stride, output_padding=0, dilation=dilation, padding=dilation)

        self.norm2 = nn.BatchNorm2d(num_features=out_channels)
        self.relu = nn.ReLU()
        self.upsample = upsample
    def forward(self, x):
        # print('Decoder')
        residual = x.clone()
        # print('Residual Before upsample:', residual.size())
        # print('X Before conv:', x.size())
        x = self.relu(self.norm1(self.t_conv1(x)))
        # print('X After conv1:', x.size())
        x = self.relu(self.norm2(self.t_conv2(x)))

        if self.upsample:
            residual = self.upsample(residual)

        # print('Residual After upsample:', residual.size())
        # print('X After conv:', x.size())
        x = x + residual
        x = self.relu(x)

        return x


In [22]:
class ResNetDecoder(nn.Module):
    def __init__(self, num_blocks, ResidualBlock, dilations, num_classes,
                 encoder_out_channels=128, layer4_out_channels=16):
        super(ResNetDecoder, self).__init__()
        self.layer_in_channels = encoder_out_channels

        self.layer1 = self._make_layer(ResidualBlock, num_blocks[3], dilations[3], out_channels=64)
        self.layer2 = self._make_layer(ResidualBlock, num_blocks[2], dilations[2], out_channels=32, stride=1)
        self.layer3 = self._make_layer(ResidualBlock, num_blocks[1], dilations[1], out_channels=16, stride=1)
        self.layer4 = self._make_layer(ResidualBlock, num_blocks[0], dilations[0], out_channels=16, stride=1)

        self.t_conv = nn.ConvTranspose2d(in_channels=layer4_out_channels, out_channels=num_classes,\
                    kernel_size=3, stride=2, padding=1, output_padding=1)
        self.norm1 = nn.BatchNorm2d(num_features=num_classes)
        self.unpool = nn.MaxUnpool2d(kernel_size=2, stride=2)
        self.relu = nn.ReLU()


    def _make_layer(self, ResidualBlock, num_blocks, dilation, out_channels, stride=1):
        upsample = None
        if stride != 1 or self.layer_in_channels != out_channels * ResidualBlock.expansion:
            upsample = nn.Sequential(
                nn.ConvTranspose2d(self.layer_in_channels, out_channels * ResidualBlock.expansion, stride=stride, kernel_size=1),
                nn.BatchNorm2d(out_channels * ResidualBlock.expansion),
            )

        layer = []
        layer.append(ResidualBlock(self.layer_in_channels, out_channels, dilation, stride, upsample))
        self.layer_in_channels = out_channels * ResidualBlock.expansion

        for i in range(1, num_blocks):
            layer.append(ResidualBlock(self.layer_in_channels, out_channels, dilation, stride))

        return nn.Sequential(*layer)

    # def forward(self, x, ind_1, ind_2, ind_3, ind_4, ind_5):
    def forward(self, x, ind_1, ind_2, ind_3, ind_4, p1_shape, p2_shape, p3_shape, p4_shape):
        x = self.unpool(x, ind_4, output_size=p4_shape)
        x = self.layer1(x)
        # x = self.unpool(x, ind_5)
        x = self.unpool(x, ind_3, output_size=p3_shape)
        x = self.layer2(x)
        x = self.unpool(x, ind_2, output_size=p2_shape)
        x = self.layer3(x)
        x = self.unpool(x, ind_1, output_size=p1_shape)
        # x = self.unpool(x, ind_3)
        x = self.layer4(x)
        # x = self.unpool(x, ind_2)
        # print('AFTER LAYER4 DECODER:', x.size())
        x = self.relu(self.norm1(self.t_conv(x)))
        # print('AFTER LAST CONV DECODER:', x.size())
        # x = self.unpool(x, ind_1, output_size=pool1_shape)



        return x

In [29]:
class ImageSegmentation(nn.Module):
    def __init__(self, num_blocks, dilations, task):
        super(ImageSegmentation, self).__init__()
        num_classes = 7 if task == 'clothing' else 2
        self.encoder = ResNetEncoder(num_blocks, ResidualBlockEncoder, dilations, img_channels=3)
        self.decoder = ResNetDecoder(num_blocks, ResidualBlockDecoder, dilations,
                                     encoder_out_channels=128, layer4_out_channels=16, num_classes=num_classes)
        # self.pool = nn.AdaptiveAvgPool2d(None)

    def forward(self, x):

        # x, ind_1, ind_2, ind_3, ind_4, ind_5 = self.encoder(x)
        x, ind_1, ind_2, ind_3, ind_4, p1_shape, p2_shape, p3_shape, p4_shape = self.encoder(x)
        # print('AFTER ENCODER: ', x.size())
        # x = self.pool(x)
        # x = self.decoder(x, ind_1, ind_2, ind_3, ind_4, ind_5)
        x = self.decoder(x, ind_1, ind_2, ind_3, ind_4, p1_shape, p2_shape, p3_shape, p4_shape)

        return x


In [15]:
def get_accuracy(model, data):
    # note: why should we use a larger batch size here?
    # We do not need to compute paramater updates, thus a smaller batch size is
    # not advantageous. A larger batch size will reduce number of iterations
    # and would allow for more parallel computations. Therefore, it would be
    # faster.
    loader = torch.utils.data.DataLoader(data, batch_size=256)

    model.eval() # annotate model for evaluation (why do we need to do this?)
    # some layers act differently in training vs testing, e.g: BatchNorm,
    # Dropout. Need to "turn them off" when calculating accuracy.
    correct = 0
    total = 0
    N = 0

    if torch.cuda.is_available():
        model.cuda()
    for imgs, labels in loader:
        if torch.cuda.is_available():
            imgs = imgs.cuda()
            labels = labels.cuda()
        output = model(imgs)
        pred = output.max(1, keepdim=True)[1]
        correct += pred.eq(labels.view_as(pred)).sum().item()
        N += imgs.shape[0]

    return correct/N # TODO: return the model accuracy

In [24]:
def train(train_data, valid_data, model, checkpoint_path, num_epochs=10, learning_rate=1e-3, batch_size=32,
          weight_decay=0):
    train = torch.utils.data.DataLoader(train_data, batch_size=batch_size,\
                                        shuffle=True)

    criterion = nn.CrossEntropyLoss()
    if not os.path.exists(checkpoint_path):
        os.mkdir(checkpoint_path)
    epoch = 0
    train_accs, train_losses = [], []
    val_accs = []
    iters = []
    n = 0
    epochs = []
    if torch.cuda.is_available():
        model.cuda()
        criterion.cuda()
    while epoch < num_epochs:
        learning_rate = learning_rate
        optimizer = optim.Adam(model.parameters(),
                lr=learning_rate,
                weight_decay=weight_decay)
        for imgs, labels in train:

            if imgs.shape[0] < batch_size:
                # Last batch will have less images since it will be the
                # remainder of Total images / batch_size.

                # Adjust the learning rate of the last batch by
                # (size(last_batch) / batch_size) to account
                # for the smaller size.

                learning_rate = learning_rate * (imgs.shape[0] / batch_size)
                optimizer = optim.Adam(model.parameters(),
                                        lr=learning_rate,
                                        weight_decay=weight_decay)
            if torch.cuda.is_available():
                imgs = imgs.cuda()
                labels = labels.cuda()

            outputs = model(imgs)
            labels = labels.squeeze(dim=1)

            # print('OUTPUTS:', outputs.size())
            # print('LABELS:', labels.type(torch.LongTensor).size())
            print('output:', outputs)
            loss = criterion(outputs, labels.type(torch.LongTensor))
            torch.autograd.set_detect_anomaly(True)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            n += 1
            iters.append(n)
            # average loss
            train_losses.append(float(loss)/batch_size)



        # end of epoch
        epoch += 1
        epochs.append(epoch)
        train_acc = get_accuracy(model, train_data)
        train_accs.append(train_acc)


        val_acc = get_accuracy(model, valid_data)
        val_accs.append(val_acc)


        print("Train accuracy: ", train_acc)
        print("Validation accuracy: ", val_acc)

        checkpoint_path = '/content/gdrive/My Drive/CSC420/A2/ckptBatch-{}.pk'

        torch.save(model.state_dict(), checkpoint_path.format(epoch))

    return iters, train_losses, epochs, train_accs, val_accs


In [17]:
person_checkpoint = "/content/gdrive/MyDrive/CSC420_Final_Project/project3/person_checkpoint"
clothing_checkpoint = "/content/gdrive/MyDrive/CSC420_Final_Project/project3/person_checkpoint"

In [18]:
img_size = train_data_person.__getitem__(1)[1].size()
num_blocks = [2, 2, 4, 8]
dilations = [1, 1, 2, 4]

<ipython-input-8-fe6acb125cd4>:23: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return (torch.tensor(img, dtype=torch.float32), torch.tensor(label, dtype=torch.float32))


In [26]:
task = 'person'
model = ImageSegmentation(num_blocks=num_blocks, dilations=dilations, task=task)

In [ ]:
iters, train_losses, epochs, train_accs, val_accs = train(train_data_person,
                                                          val_data_person, model,
                                                          num_epochs=10,
                                                          learning_rate=1e-4,
                                                          batch_size=2,
                                                          weight_decay=0,
                                                          checkpoint_path=person_checkpoint)

In [30]:
task = 'clothing'
model = ImageSegmentation(num_blocks=num_blocks, dilations=dilations, task=task)

In [ ]:
iters, train_losses, epochs, train_accs, val_accs = train(train_data_clothing,
                                                          val_data_clothing, model,
                                                          num_epochs=10,
                                                          learning_rate=1e-4,
                                                          batch_size=2,
                                                          weight_decay=0,
                                                          checkpoint_path=clothing_checkpoint)

<ipython-input-8-fe6acb125cd4>:23: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return (torch.tensor(img, dtype=torch.float32), torch.tensor(label, dtype=torch.float32))


output: tensor([[[[0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          [0.0000e+00, 0.0000e+00, 4.7581e-01,  ..., 0.0000e+00,
           9.6910e-02, 2.0795e-01],
          [0.0000e+00, 0.0000e+00, 1.9050e-01,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          ...,
          [1.6277e-01, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           6.8683e-01, 4.3081e-01],
          [3.1263e-03, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          [0.0000e+00, 4.3648e-01, 0.0000e+00,  ..., 1.0453e-01,
           0.0000e+00, 3.8300e-01]],

         [[0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          [0.0000e+00, 4.1882e-01, 0.0000e+00,  ..., 1.8569e+00,
           0.0000e+00, 0.0000e+00],
          [0.0000e+00, 7.2007e-01, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 5.3326e-02],
          ...,
          [1.4781e-01, 3.1379e-01, 7.6937e-01,  ...